In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For pandas visualization
pd.options.display.max_columns=30
styles = [dict(selector="th", props=[("font-size", "200%"),("text-align", "center")]),
          dict(selector="td", props=[("font-size", "150%")])]
feats_viz = ['sample_peak', 'volt_amp', 'period', 'time_rdsym', 'time_ptsym', 'is_burst']
names = ['sine', 'sawtooth', 'burst', 'nonstationary']
colors = sns.color_palette()[:len(names)]

burst_kwargs = {'amplitude_fraction_threshold': .3,
                'amplitude_consistency_threshold': .4,
                'period_consistency_threshold': .5,
                'monotonicity_threshold': .8,
                'N_cycles_min': 3}

np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')


def plot_psds(f, psd_white, psd_delta):
    plt.figure(figsize=(5,5))
    plt.semilogy(f, psd_white, 'k', label='White noise', linewidth=3, alpha=.8)
    plt.semilogy(f, psd_delta, 'r', label='Delta function', linewidth=3, alpha=.8)
    plt.xlabel('Frequency', size=20)
    plt.ylabel('Power', size=20)
    plt.xticks([])
    plt.yticks([])
    plt.xlim((1, 40))
    plt.ylim((10**-6, 10))
    plt.legend(fontsize=20)
    plt.show()
    
    
def plot_signals(Fs, white_noise, delta_fn):
    t = np.arange(0, len(white_noise)/Fs, 1/Fs)
    
    plt.figure(figsize=(10,5))
    plt.subplot(2,1,1)
    plt.plot(t, white_noise, 'k', linewidth=3)
    plt.title('White noise', size=20)
    plt.xticks([])
    plt.yticks([])
    
    plt.subplot(2,1,2)
    plt.plot(t, delta_fn, 'r', linewidth=3)
    plt.title('Delta function', size=20)
    plt.xticks([])
    plt.yticks([])
    plt.show()


def simulate_oscillators(freq, T, Fs):
    from neurodsp import sim_bursty_oscillator
    np.random.seed(1)
    xs = [0] * 4
    xs[0] = sim_bursty_oscillator(freq, T, Fs, prob_enter_burst=1, prob_leave_burst=0,
                                  cycle_features={'amp_mean': 1, 'amp_std': 0, 'amp_burst_std': 0,
                                                  'period_std': 0, 'period_burst_std': 0,
                                                  'rdsym_mean': .5, 'rdsym_std': 0, 'rdsym_burst_std': 0})
    xs[1] = sim_bursty_oscillator(freq, T, Fs, prob_enter_burst=1, prob_leave_burst=0,
                                  cycle_features={'amp_mean': 1, 'amp_std': 0, 'amp_burst_std': 0,
                                                  'period_std': 0, 'period_burst_std': 0,
                                                  'rdsym_mean': .2, 'rdsym_std': 0, 'rdsym_burst_std': 0})
    xs[2] = sim_bursty_oscillator(freq, T, Fs, prob_enter_burst=.25, prob_leave_burst=.25,
                                  cycle_features={'amp_mean': 1.5, 'amp_std': 0, 'amp_burst_std': 0,
                                                  'period_std': 0, 'period_burst_std': 0,
                                                  'rdsym_mean': .5, 'rdsym_std': 0, 'rdsym_burst_std': 0})
    xs[3] = sim_bursty_oscillator(freq, T, Fs, prob_enter_burst=1, prob_leave_burst=0,
                                  cycle_features={'amp_mean': 1.3, 'amp_std': .6, 'amp_burst_std': 0,
                                                  'period_std': 20, 'period_burst_std': 0,
                                                  'rdsym_mean': .5, 'rdsym_std': .12, 'rdsym_burst_std': 0})
    for x in xs:
        x += np.random.randn(len(x))*.001
    return xs


def cycle_by_cycle_quartet_analysis(xs):
    from bycycle.features import compute_features
    dfs = []
    f_range = (8, 12)
    ampthresh = (.5, 1)
    for i, (x_temp, name) in enumerate(zip(xs, names)):
        df = compute_features(x_temp, 1000, f_range, center_extrema='T',
                              burst_detection_method = 'amp',
                              burst_detection_kwargs = {'amp_threshes': ampthresh,

                                                        'filter_kwargs': {'N_seconds': .3}})
        df['signal'] = name
        dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)

    df_plt1 = df[df['is_burst']][['time_rdsym', 'volt_amp', 'period', 'signal']].set_index(
        'signal').stack().reset_index().rename(
        columns={'level_1': 'feature', 0:'value'})
    df_plt2 = df[['is_burst', 'signal']].set_index(
        'signal').stack().reset_index().rename(
        columns={'level_1': 'feature', 0:'value'})
    df_plt = pd.concat([df_plt1, df_plt2])

    df_plt.replace({'time_rdsym': 'rise-decay symmetry',
                    'is_burst': 'oscillation presence',
                    'volt_amp': 'amplitude'}, inplace=True)

    g = sns.catplot(x='value', y='signal', col='feature', ci='sd',
                    sharex=False, hue_order=names,
                    col_order= ['oscillation presence', 'amplitude', 'period', 'rise-decay symmetry'],
                    data=df_plt, kind='bar')

    (g.set_axis_labels("", "")
     .set_yticklabels(names, size=20)
     .set_titles("{col_name}", size=20)
     .set_xticklabels([])
     .despine(left=True, right=True, bottom=True))


def plot_oscillators(xs):
    plt.figure(figsize=(12,6))
    for i, color, name in zip(range(len(xs)), colors, names):
        plt.subplot(4,1,i+1)
        plt.plot(xs[i], color=color, linewidth=3)
        plt.ylim((-4.2, 4.2))
        plt.xlim((500, 2500))
        plt.xticks([])
        plt.yticks([]) 
        plt.ylabel(name, color=color, size=15)
        

def plot_oscillator_spectra(xs):
    psds = [0] * 4
    for i, x in enumerate(xs):
        f, psds[i] = spectral.psd(xs[i], Fs=1000)

    flims = (0, 30)
    fidxs = np.logical_and(f >= flims[0], f < flims[1])
    plt.figure(figsize=(6,6))
    for psd, name, color in zip(psds, names, colors):
        plt.semilogy(f[fidxs], psd[fidxs], linewidth=3, label=name, color=color)
    plt.legend(fontsize=15, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.fill_between(f[fidxs], 10**-100, 10*100,
                     where=np.logical_and(f[fidxs]>=8, f[fidxs]<=12),
                     interpolate=True, facecolor='black', alpha=0.2)
    plt.yticks([])
    plt.ylim((10**-10, 10**0))
    plt.ylabel('Power', size=20)
    plt.xlabel('Frequency (Hz)', size=20)
    plt.xticks([0, 10, 20, 30], size=20)
    plt.xlim((0, 30))
    
    
def plot_signals_filt(x_raw, x):
    plt.figure(figsize=(12,3))
    plt.plot(x_raw, 'k', label='raw', linewidth=3)
    plt.plot(x+1, 'r', label='lowpass', linewidth=3)
    plt.xticks([])
    plt.yticks([])
    plt.legend(fontsize=15, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlim(5900, 7200)


def plot_power_comparison():
    plt.figure(figsize=(5,5))
    df_plt = pd.DataFrame({'name':names, 'power':[1]*len(names)})
    sns.barplot(x='power', y='name', data=df_plt)
    plt.xlim((0,2))
    plt.xticks([])
    plt.yticks(size=20)
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Oscillatory power', size=20)


Exporting to html: `jupyter nbconvert --to slides SfN\ nanosymposium\ presentation.ipynb --post serve`

Exporting to pdf instructions: https://github.com/damianavila/RISE/issues/127#issuecomment-159355863

# Cycle-by-cycle analysis of neural oscillations
### Python package: `bycycle`
#### Scott Cole
#### Voytek Lab
#### SfN 2018


In [ ]:
Fs = 100
white_noise = np.random.randn(1000)
delta_fn = np.array([0 if i != 500 else
    np.sqrt(np.sum(white_noise**2) / 2) for i in range(1000)])

plot_signals(Fs, white_noise, delta_fn)

## Ambiguous spectral representations

In [ ]:
from neurodsp import spectral
f, psd_white = spectral.compute_spectrum(white_noise, Fs)
_, psd_delta = spectral.compute_spectrum(delta_fn, Fs)

plot_psds(f, psd_white, psd_delta)

## Ambiguous spectral representations: oscillations

<center>

<img src="img/quartet.png" width="800" align="center">
</center>

## Ambiguous spectral representations: oscillations

<center>

<img src="img/quartet_and_powers.png" width="1200" align="center">
</center>

## Cycle-by-cycle parametrization

<center>

<img src="img/quartet_sawtooth.png" width="1000" align="center">
</center>

## Cycle-by-cycle parametrization

<center>

<img src="img/quartet_burst.png" width="1000" align="center">
</center>

## Cycle-by-cycle parametrization

<center>

<img src="img/quartet_nonstationary.png" width="1000" align="center">
</center>

In [ ]:
# xs = simulate_oscillators(freq=10, T=10, Fs=1000)
# plot_oscillators(xs)
# plot_oscillator_spectra(xs)
# plot_power_comparison()
# cycle_by_cycle_quartet_analysis(xs)

## Cycle-by-cycle analysis algorithm:

### 1. Segmentation

<center>

<img src="img/segmentation.png" width="1200" align="center">
</center>

## Cycle-by-cycle analysis algorithm:
### 2. Cycle features

<center>

<img src="img/features_schematic.png" width="800" align="center">
</center>

## Cycle-by-cycle analysis algorithm
### 3. Oscillation detection

<center>

<img src="img/oscillation_detection.png" width="1200" align="center">
</center>

# Applying `bycycle` to your data

In [ ]:
# Load data
x_raw = np.load('../tutorials/data/sim_bursting.npy')
Fs = 1000
f_range = (8, 12)

## `bycycle`: preprocessing

In [ ]:
# Apply lowpass filter for extrema localization
from neurodsp.filt import filter_signal
x = filter_signal(x_raw, Fs, 'lowpass', 30, n_seconds=.2, remove_edge_artifacts=False)

plot_signals_filt(x_raw, x)

## `bycycle`: cycle feature computation

In [ ]:
from bycycle.features import compute_features
df = compute_features(x, Fs, f_range, burst_detection_kwargs=burst_kwargs)
df[feats_viz].head().style.set_table_styles(styles)

## `bycycle`: oscillation detection

In [ ]:
from bycycle.burst import plot_burst_detect_params
plot_burst_detect_params(x, Fs, df, burst_kwargs, plot_only_result=True)

# Why use `bycycle`?

### 1. Explicitly parametrizes waveform shape and oscillation presence

"instantaneous" amplitude and frequency **confounds with** oscillation presence

(Cole & Voytek, *bioRxiv*, 2018a)

Phase-amplitude coupling **confounds with** waveform shape

(Cole et al., *J Neuro*, 2017)

### 2. New analysis possibilities
### (Cole & Voytek, *bioRxiv*, 2018b)

* Theta asymmetry ~ burst duration

* Theta asymmetry ~ CA1 neuron firing patterns

* Theta presence ~ CA1 neuron firing patterns

![](img/acknowledge.png)